<a href="https://colab.research.google.com/github/hoavu-cs/notebooks/blob/main/tiny_english_stories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
import math
from tqdm import tqdm
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, BertTokenizer
import gc


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00


In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
!ls "/content/drive/My Drive/datasets/NLP/tiny_vietnamese_stories"
data_path = "/content/drive/My Drive/datasets/NLP/tiny_vietnamese_stories"


Mounted at /content/drive
1000000.parquet  2200000.parquet  english_tiny_stories.pth
1200000.parquet  2400000.parquet  merges.txt
1400000.parquet  2600000.parquet  tiny_stories.csv
1600000.parquet  400000.parquet   vietnamese_tiny_stories.pth
1800000.parquet  600000.parquet   vocab.json
2000000.parquet  800000.parquet
200000.parquet	 bpe_train.txt


In [4]:
file_dir = data_path
filename = '1000000.parquet'

# Load and concatenate data from all Parquet files
df = pd.read_parquet(os.path.join(file_dir, filename))
text = '\n'.join(df['en'])
with open(os.path.join(file_dir, 'bpe_train.txt'), 'w', encoding='utf-8') as f:
    f.write(text)


In [5]:
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=[os.path.join(file_dir, 'bpe_train.txt')], vocab_size=5000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# 3. Save the tokenizer (optional)
# You can save the trained tokenizer to reuse later
tokenizer.save_model(data_path)

# 5. Using processors for compatibility (optional)
# Configure the tokenizer to output the special tokens needed for models like BERT.
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)


In [4]:
vocab_path = tokenizer_train_path = os.path.join(data_path, "vocab.json")
merge_path = tokenizer_train_path = os.path.join(data_path, "merges.txt")

tokenizer = ByteLevelBPETokenizer(
    vocab=vocab_path,
    merges=merge_path
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)


In [5]:
# 4. Encode a text string
output = tokenizer.encode("I love cats but I love dogs more.", add_special_tokens=True)
print("Encoded string: ", output.ids)  # output.ids is the tokenized representation
print("Decoded string: ", tokenizer.decode(output.ids))  # decoding back to the original string

Encoded string:  [0, 45, 872, 2875, 413, 342, 872, 2399, 680, 18, 2]
Decoded string:  <s>I love cats but I love dogs more.</s>


In [ ]:
# List of Parquet files to merge
# file_dir = data_path
# file_names = [f for f in os.listdir(file_dir) if f.endswith('.parquet')]


# dataframes = [pd.read_parquet(os.path.join(file_dir, file_name)) for file_name in file_names]
# merged_dataframe = pd.concat(dataframes, ignore_index=True)


# print(merged_dataframe.head())
# merged_dataframe.to_parquet("merged_data.parquet")
# merge_file = os.path.join(file_dir, 'tiny_stories.csv')
# merged_dataframe.to_csv(merge_file)


In [6]:
merge_file = os.path.join(data_path, 'tiny_stories.csv')

data = pd.read_csv(merge_file)


In [7]:
N = len(data)
data.dropna(inplace=True)
train_size = int(0.9 * N)
train_data = data[:train_size]
val_data = data[train_size:]


In [8]:
BLOCK_SIZE = 512
VOCAB_SIZE = len(tokenizer.get_vocab())
print('Vocab size is ', VOCAB_SIZE)
#UNK_ID = tokenizer.convert_tokens_to_ids('[UNK]')

Vocab size is  5000


In [9]:
end_id = tokenizer.token_to_id("</s>")
start_id = tokenizer.token_to_id("<unk>")
pad_id = tokenizer.token_to_id("<pad>")
unk_id = tokenizer.token_to_id("<unk>")

In [12]:
def get_batch(data, block_size, batch_size):
    x = torch.zeros((batch_size, block_size), dtype=torch.long)
    y = torch.zeros((batch_size, block_size), dtype=torch.long)
    samples = data['en'].sample(n=batch_size)

    for i, sample in enumerate(samples):
        summary_ids = tokenizer.encode(sample).ids
        #summary_ids = [id for id in summary_ids if id != UNK_ID]
        if len(summary_ids) < block_size + 2:
            summary_ids = summary_ids + [pad_id] * (block_size + 2 - len(summary_ids))
        random_start = random.randint(0, len(summary_ids) - block_size - 2)
        x[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start:random_start + block_size], dtype=torch.long)
        y[i, :len(summary_ids)] = torch.tensor(summary_ids[random_start + 1:random_start + block_size + 1], dtype=torch.long)

    return x, y

a, b = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=1)
print(a.shape, b.shape)
print(tokenizer.decode(a[0].tolist()))
print(tokenizer.decode(b[0].tolist()))

torch.Size([1, 512]) torch.Size([1, 512])
<s>Once upon a time, there was an ordinary boy named Tom. Tom had a big toe that would remind him to do things. One day, Tom's toe told him to go to the park.
At the park, Tom saw a little dog. The dog was lost and sad. Tom's toe reminded him to help the dog find its way home. So, Tom and the dog walked around the park to look for the dog's house.
After a while, they found the dog's house. The dog was very happy to be home. Tom's ordinary toe had helped him do a good thing. Tom and the dog became best friends and played together every day.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
del model  # delete the tensor variable
torch.cuda.empty_cache()  # clear unused memory in PyTorch
gc.collect()  # call Python garbage collector

102

In [11]:
N_EMB = 800
N_LAYERS = 4
N_HEADS = 5
DROPOUT = 0.2


def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

def generate_square_subsequent_mask(sz):
    mask = (torch.tril(torch.ones(sz, sz)) == 1).float()
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def get_sine_position_encodings(length, dim):
    pos = torch.arange(length, dtype=torch.float32).reshape(-1, 1)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(math.log(10000.0) / dim))
    pos_encodings = torch.zeros(length, dim)
    pos_encodings[:, 0::2] = torch.sin(pos * div_term)
    pos_encodings[:, 1::2] = torch.cos(pos * div_term)
    return pos_encodings

class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb, block_size, n_layers, n_heads, dropout=0.2):
        super(LanguageModel, self).__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_encodings = get_sine_position_encodings(block_size, n_emb).to(device)

        encoder_layer = nn.TransformerEncoderLayer(d_model=n_emb, nhead=n_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.feed_forward = nn.Sequential(
            nn.Linear(n_emb, 2 * n_emb),
            nn.ReLU(),
            nn.Linear(2 * n_emb, n_emb)
        )

        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_encodings[:T, :].unsqueeze(0).expand(B, -1, -1)

        x = token_emb + position_emb
        x_transform = x.clone()
        mask = generate_square_subsequent_mask(T).to(device)

        x_transform = self.transformer_encoder(x_transform.permute(1, 0, 2), mask=mask)
        x_transform = x_transform.permute(1, 0, 2)
        x = x + x_transform

        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None


    def generate(self, idx, max_new_tokens, block_size, temperature=1.0, stop_token=False):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self.forward(idx_cond)

            # Scale logits by the temperature
            logits = logits[:, -1, :] / temperature

            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
            if stop_token and idx_new.item() == end_id:
                break
        return idx

# Create model, optimizer
model = LanguageModel(vocab_size=VOCAB_SIZE, block_size=BLOCK_SIZE, n_emb=N_EMB, n_layers=N_LAYERS, \
    n_heads=N_HEADS, dropout=DROPOUT).to(device)

print(f'Number of parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}')


Number of parameters 33951592


In [13]:
model_path = os.path.join(data_path, "english_tiny_stories.pth")
model = torch.load(model_path)

In [ ]:
EARLY_STOP = 50
N_EPOCHS = 5000
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

last_val_loss = 1e9
early_stop = EARLY_STOP

for steps in range(N_EPOCHS):
    model.train()
    xb, yb = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Delete xb, yb and free GPU memory
    del xb, yb
    torch.cuda.empty_cache()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(model, val_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
        print('Validation loss:', val_loss)
        if val_loss >= last_val_loss:
            early_stop -= 1
            if early_stop == 0:
                print('Early stop!')
                break
        else:
            early_stop = EARLY_STOP
            last_val_loss = val_loss

In [29]:
model_path = os.path.join(data_path, "english_tiny_stories.pth")
torch.save(model, model_path)

In [19]:
starting_tokens = 'Alice went to a wonderful park. She met a dog that can speak English.'

encoded_start = tokenizer.encode(starting_tokens).ids
encoded_start.pop(-1)
len_starting_tokens = len(encoded_start)

idx = torch.tensor(encoded_start).reshape(1, len_starting_tokens).to(device)
model.eval()
N_SAMPLES = 100
for _ in range(N_SAMPLES):
    generation = model.generate(idx, max_new_tokens=2000, block_size=BLOCK_SIZE, temperature=0.7, stop_token=True)[0].tolist()
    story = tokenizer.decode(generation)
    print('Story ', _ + 1, ':')
    story = story.replace('</s>', '')
    story = story.replace('<s>', '')
    print(story)
    print('\n \n')






Story  1 :
Alice went to a wonderful park. She met a dog that can speak English. She and the dog were very excited.
"Look at the park!" said Jenny.
"What is that?" asked the dog.
"That is a wild bird," said Jenny.
The dog seemed to answer.
Jenny and the dog played together and laughed. They had so much fun. They laughed and laughed.
Then, something unexpected happened. A big wind came and blew Johnny's own wings away. Alice and the dog hid behind a tree.
The dog was scared and sad. Jenny and the dog were very sad. They could not find their way back home.
The dog was not nice. He said, "Don't worry, I will help you find your home."
The dog opened her eyes and saw Jenny's eyes widened. Jenny and the dog were happy again, and they went home with their new own journey.

 

Story  2 :
Alice went to a wonderful park. She met a dog that can speak English. She asked the dog, "Can you help me find a friend?" The dog said, "Yes, I can help you."
Alice and the dog went to the park. They found a b